<a href="https://colab.research.google.com/github/ipejun-ai/m5-accuracy/blob/master/ModelWithImprovedAccuracy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<hr>
1. Thanks to Konstantin Yakovlev for excellent Kernal, where i got the help
2. We have done some improvement as per previous model and accuracy have been increase to .47450
3. Please upvote the Kernal in case it seems helpful for you all

In [0]:
# General imports
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

# custom imports
from multiprocessing import Pool        # Multiprocess Runs

warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
DIRPATH="/content/gdrive/My Drive/kaggle/"
#DIRPATH="C:/Users/peiju/Documents/Study/kaggle/m5-forecasting-accuracy/"

In [0]:

def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)

    
## Multiprocess Runs
def df_parallelize_run(func, t_split):
    num_cores = np.min([N_CORES,len(t_split)])
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, t_split), axis=1)
    pool.close()
    pool.join()
    return df

In [0]:

def get_data_by_store(store):
    
    # Read and contact basic feature
    df = pd.concat([pd.read_pickle(BASE),
                    pd.read_pickle(PRICE).iloc[:,2:],
                    pd.read_pickle(CALENDAR).iloc[:,2:]],
                    axis=1)
    
    df = df[df['store_id']==store]
    df2 = pd.read_pickle(MEAN_ENC)[mean_features]
    df2 = df2[df2.index.isin(df.index)]
    
    df3 = pd.read_pickle(LAGS).iloc[:,3:]
    df3 = df3[df3.index.isin(df.index)]
    
    df = pd.concat([df, df2], axis=1)
    del df2 # to not reach memory limit 
    
    df = pd.concat([df, df3], axis=1)
    del df3 # to not reach memory limit 
    
    # Create features list
    features = [col for col in list(df) if col not in remove_features]
    df = df[['id','d',TARGET]+features]
    
    # Skipping first n rows
    df = df[df['d']>=START_TRAIN].reset_index(drop=True)
    
    return df, features

# Recombine Test set after training
def get_base_test():
    base_test = pd.DataFrame()

    for store_id in STORES_IDS:
        temp_df = pd.read_pickle('test_'+store_id+'.pkl')
        #temp_df = pd.read_pickle('test_'+store_id+'.pkl')
        temp_df['store_id'] = store_id
        base_test = pd.concat([base_test, temp_df]).reset_index(drop=True)
    
    return base_test


def make_lag(LAG_DAY):
    lag_df = base_test[['id','d',TARGET]]
    col_name = 'sales_lag_'+str(LAG_DAY)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(LAG_DAY)).astype(np.float16)
    return lag_df[[col_name]]


def make_lag_roll(LAG_DAY):
    shift_day = LAG_DAY[0]
    roll_wind = LAG_DAY[1]
    lag_df = base_test[['id','d',TARGET]]
    col_name = 'rolling_mean_tmp_'+str(shift_day)+'_'+str(roll_wind)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(shift_day).rolling(roll_wind).mean())
    return lag_df[[col_name]]

sell_prices.csv

In [0]:
########################### Model params
import lightgbm as lgb
lgb_params = {
                    'boosting_type': 'gbdt',
                    'objective': 'tweedie',
                    'tweedie_variance_power': 1.1,
                    'metric': 'rmse',
                    'subsample': 0.5,
                    'subsample_freq': 1,
                    'learning_rate': 0.03,
                    'num_leaves': 2**11-1,
                    'min_data_in_leaf': 2**12-1,
                    'feature_fraction': 0.5,
                    'max_bin': 100,
                    'n_estimators': 1400,
                    'boost_from_average': False,
                    'verbose': -1,
                } 




In [0]:
#prices.info(memory_usage="deep")

In [0]:

#prices.memory_usage(deep=True) * 1e-6

In [0]:
#prices.memory_usage(deep=True).sum() * 1e-6

In [0]:
########################### Vars
#################################################################################
VER = 2                          # Our model version
SEED = 42                        
seed_everything(SEED)            
lgb_params['seed'] = SEED       
N_CORES = psutil.cpu_count()     

#LIMITS and const
TARGET      = 'sales'           
START_TRAIN = 0                  
END_TRAIN   = 1913               
P_HORIZON   = 28                
USE_AUX     = False           

#FEATURES to remove
## These features lead to overfit
remove_features = ['id','state_id','store_id',
                   'date','wm_yr_wk','d',TARGET]
mean_features   = ['enc_cat_id_mean','enc_cat_id_std',
                   'enc_dept_id_mean','enc_dept_id_std',
                   'enc_item_id_mean','enc_item_id_std'] 


ORIGINAL = DIRPATH
BASE     = DIRPATH+'/output/m5-simple-fe/grid_part_1.pkl'
PRICE    = DIRPATH+'/output/m5-simple-fe/grid_part_2.pkl'
CALENDAR = DIRPATH+'/output/m5-simple-fe/grid_part_3.pkl'
LAGS     = DIRPATH+'/output/m5-lags-features/lags_df_28.pkl'
MEAN_ENC = DIRPATH+'/output/m5-custom-features/mean_encoding_df.pkl'
# AUX(pretrained) Models paths
AUX_MODELS = DIRPATH+'/output/m5modelwithestimator1300/'



STORES_IDS = pd.read_csv(ORIGINAL+'sales_train_validation.csv')['store_id']
STORES_IDS = list(STORES_IDS.unique())




#SPLITS for lags creation
SHIFT_DAY  = 28
N_LAGS     = 15
LAGS_SPLIT = [col for col in range(SHIFT_DAY,SHIFT_DAY+N_LAGS)]
ROLS_SPLIT = []
for i in [1,7,14]:
    for j in [7,14,30,60]:
        ROLS_SPLIT.append([i,j])

In [11]:
STORES_IDS

['CA_1',
 'CA_2',
 'CA_3',
 'CA_4',
 'TX_1',
 'TX_2',
 'TX_3',
 'WI_1',
 'WI_2',
 'WI_3']

In [0]:
gc.collect()

0

**We can see above table is taking a lot of space which is not at all good for further computation.Let us identify some columns which are consuming most of memory and space**

**To make this easier to read, we convert all values to megabytes**

In [0]:
#Model Training

In [0]:

if USE_AUX:
    lgb_params['n_estimators'] = 2
    


In [14]:
########################### Train Models
#################################################################################
for store_id in STORES_IDS:
    print('Train', store_id)
    
    # Get grid for current store
    grid_df, features_columns = get_data_by_store(store_id)
    
    train_mask = grid_df['d']<=END_TRAIN
    valid_mask = train_mask&(grid_df['d']>(END_TRAIN-P_HORIZON))
    preds_mask = grid_df['d']>(END_TRAIN-100)
    

    train_data = lgb.Dataset(grid_df[train_mask][features_columns], 
                       label=grid_df[train_mask][TARGET])
    train_data.save_binary('train_data.bin')
    train_data = lgb.Dataset('train_data.bin')
    
    valid_data = lgb.Dataset(grid_df[valid_mask][features_columns], 
                       label=grid_df[valid_mask][TARGET])
    
    grid_df = grid_df[preds_mask].reset_index(drop=True)
    keep_cols = [col for col in list(grid_df) if '_tmp_' not in col]
    grid_df = grid_df[keep_cols]
    grid_df.to_pickle('test_'+store_id+'.pkl')
    del grid_df
    
    # Launch seeder again to make lgb training 100% deterministic
    # with each "code line" np.random "evolves" 
    # so we need (may want) to "reset" it
    seed_everything(SEED)
    estimator = lgb.train(lgb_params,
                          train_data,
                          valid_sets = [valid_data],
                          verbose_eval = 100,
                          )
    
    model_name = AUX_MODELS+'lgb_model_'+store_id+'_v'+str(VER)+'.bin'
    pickle.dump(estimator, open(model_name, 'wb'))

    !rm train_data.bin
    del train_data, valid_data, estimator
    gc.collect()
    
    # "Keep" models features for predictions
    MODEL_FEATURES = features_columns

Train CA_1
[100]	valid_0's rmse: 2.02301
[200]	valid_0's rmse: 2.00225
[300]	valid_0's rmse: 1.99299
[400]	valid_0's rmse: 1.98575
[500]	valid_0's rmse: 1.9791
[600]	valid_0's rmse: 1.97285
[700]	valid_0's rmse: 1.96813
[800]	valid_0's rmse: 1.96247
[900]	valid_0's rmse: 1.95811
[1000]	valid_0's rmse: 1.95347
[1100]	valid_0's rmse: 1.94855
[1200]	valid_0's rmse: 1.94369
[1300]	valid_0's rmse: 1.93954
[1400]	valid_0's rmse: 1.93483
Train CA_2
[100]	valid_0's rmse: 1.89295
[200]	valid_0's rmse: 1.84982
[300]	valid_0's rmse: 1.83925
[400]	valid_0's rmse: 1.83126
[500]	valid_0's rmse: 1.82514
[600]	valid_0's rmse: 1.81947
[700]	valid_0's rmse: 1.81371
[800]	valid_0's rmse: 1.80924
[900]	valid_0's rmse: 1.80469
[1000]	valid_0's rmse: 1.79984
[1100]	valid_0's rmse: 1.79584
[1200]	valid_0's rmse: 1.7917
[1300]	valid_0's rmse: 1.78728
[1400]	valid_0's rmse: 1.7835
Train CA_3
[100]	valid_0's rmse: 2.50553
[200]	valid_0's rmse: 2.46381
[300]	valid_0's rmse: 2.43753
[400]	valid_0's rmse: 2.42363


In [0]:
#size=os.path.getsize("../input/m5-custom-features/mean_encoding_df.pkl")

In [15]:

all_preds = pd.DataFrame()
base_test = get_base_test()
main_time = time.time()

for PREDICT_DAY in range(1,29):    
    print('Predict | Day:', PREDICT_DAY)
    start_time = time.time()

    grid_df = base_test.copy()
    grid_df = pd.concat([grid_df, df_parallelize_run(make_lag_roll, ROLS_SPLIT)], axis=1)
        
    for store_id in STORES_IDS:
        
        model_path =  AUX_MODELS + 'lgb_model_'+store_id+'_v'+str(VER)+'.bin' 
        if USE_AUX:
            model_path = AUX_MODELS + model_path
        
        estimator = pickle.load(open(model_path, 'rb'))
        
        day_mask = base_test['d']==(END_TRAIN+PREDICT_DAY)
        store_mask = base_test['store_id']==store_id
        mask = (day_mask)&(store_mask)
        base_test[TARGET][mask] = estimator.predict(grid_df[mask][MODEL_FEATURES])
    
   
    temp_df = base_test[day_mask][['id',TARGET]]
    temp_df.columns = ['id','F'+str(PREDICT_DAY)]
    if 'id' in list(all_preds):
        all_preds = all_preds.merge(temp_df, on=['id'], how='left')
    else:
        all_preds = temp_df.copy()
        
    print('#'*10, ' %0.2f min round |' % ((time.time() - start_time) / 60),
                  ' %0.2f min total |' % ((time.time() - main_time) / 60),
                  ' %0.2f day sales |' % (temp_df['F'+str(PREDICT_DAY)].sum()))
    del temp_df
    
all_preds = all_preds.reset_index(drop=True)
all_preds

Predict | Day: 1
##########  0.84 min round |  0.84 min total |  37244.26 day sales |
Predict | Day: 2
##########  0.64 min round |  1.47 min total |  35260.70 day sales |
Predict | Day: 3
##########  0.64 min round |  2.11 min total |  34676.89 day sales |
Predict | Day: 4
##########  0.64 min round |  2.75 min total |  35292.38 day sales |
Predict | Day: 5
##########  0.64 min round |  3.39 min total |  41648.20 day sales |
Predict | Day: 6
##########  0.64 min round |  4.03 min total |  51296.06 day sales |
Predict | Day: 7
##########  0.63 min round |  4.66 min total |  53411.79 day sales |
Predict | Day: 8
##########  0.64 min round |  5.30 min total |  44072.05 day sales |
Predict | Day: 9
##########  0.63 min round |  5.92 min total |  44507.31 day sales |
Predict | Day: 10
##########  0.62 min round |  6.54 min total |  38854.35 day sales |
Predict | Day: 11
##########  0.62 min round |  7.17 min total |  40746.36 day sales |
Predict | Day: 12
##########  0.62 min round |  7.79

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.808675,0.756479,0.725383,0.783678,0.924481,1.111708,1.148330,0.857709,0.960107,0.873468,0.859240,0.880035,1.084085,0.888813,0.866033,0.793211,0.815499,0.746589,0.834873,1.150430,1.062086,0.817464,0.815657,0.788592,0.781786,0.906339,1.094172,1.041741
1,HOBBIES_1_002_CA_1_validation,0.202484,0.194100,0.170424,0.191563,0.242878,0.264590,0.302735,0.243072,0.225347,0.227898,0.220056,0.227270,0.255452,0.256408,0.204357,0.203575,0.180076,0.184335,0.215144,0.235862,0.234160,0.170684,0.162729,0.172525,0.182174,0.193815,0.254457,0.258308
2,HOBBIES_1_003_CA_1_validation,0.411106,0.399626,0.408211,0.432825,0.622228,0.692906,0.681228,0.473231,0.434298,0.455136,0.444727,0.540418,0.703361,0.540316,0.441461,0.421781,0.431653,0.472714,0.581578,0.691592,0.712611,0.501714,0.456307,0.443825,0.463924,0.619571,0.678915,0.648519
3,HOBBIES_1_004_CA_1_validation,1.734632,1.328718,1.363506,1.579711,1.931753,3.206603,3.213895,1.616610,1.441413,1.362669,1.699942,1.763482,3.060752,2.345340,1.564553,1.389965,1.450917,1.421811,1.903260,2.646641,3.304128,1.669670,1.424615,1.394853,1.456414,2.005535,2.937148,3.311513
4,HOBBIES_1_005_CA_1_validation,0.909512,0.827129,0.776101,0.831035,1.001370,1.446833,1.406213,1.001546,1.053770,0.974042,0.896584,1.055455,1.452985,1.138943,0.988968,1.023900,0.941298,0.951153,1.117601,1.528608,1.570002,1.017698,0.938244,0.943636,0.911981,1.117703,1.417977,1.422585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,0.370991,0.341365,0.327120,0.335749,0.407332,0.469080,0.481776,0.531323,0.539887,0.400359,0.512892,0.505329,0.558610,0.698402,0.520843,0.469779,0.536178,0.509028,0.489195,0.644466,0.742919,0.506512,0.439736,0.414418,0.412362,0.449285,0.518218,0.568580
30486,FOODS_3_824_WI_3_validation,0.274952,0.239064,0.217127,0.205859,0.244259,0.313284,0.318099,0.412865,0.420494,0.324734,0.342321,0.360045,0.391228,0.375376,0.382587,0.310962,0.320298,0.291690,0.277709,0.426719,0.505303,0.355385,0.252571,0.212679,0.191103,0.203620,0.286127,0.299729
30487,FOODS_3_825_WI_3_validation,0.684364,0.549088,0.483379,0.499591,0.695233,0.782563,0.936411,1.190315,1.072159,0.810478,1.065638,1.204291,1.193401,1.354643,1.305500,0.838837,1.118620,1.137112,0.939002,1.278448,1.489106,1.121092,0.747024,0.710180,0.608002,0.685441,0.867982,0.841729
30488,FOODS_3_826_WI_3_validation,1.033937,0.994073,0.864730,0.915629,0.931169,1.328792,1.170107,1.145274,1.201303,0.928736,1.186546,1.166002,1.251033,1.288964,1.264277,0.983957,1.124405,1.160044,1.056941,1.380019,1.371309,1.031803,1.031501,0.891245,0.866861,1.083007,1.143793,1.197657


In [0]:
    
submission = pd.read_csv(ORIGINAL+'sample_submission.csv')[['id']]
submission = submission.merge(all_preds, on=['id'], how='left').fillna(0)
submission.to_csv(DIRPATH+'/output/m5modelwithestimator1300//submission_v'+str(VER)+'.csv', index=False)

In [0]:
grid_df.tail()

,id,d,sales,item_id,dept_id,cat_id,release,sell_price,price_max,price_min,price_std,price_mean,price_norm,price_nunique,item_nunique,price_momentum,price_momentum_m,price_momentum_y,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,tm_d,tm_w,tm_m,tm_y,tm_wm,tm_dw,tm_w_end,enc_cat_id_mean,enc_cat_id_std,enc_dept_id_mean,enc_dept_id_std,enc_item_id_mean,enc_item_id_std,sales_lag_28,sales_lag_29,sales_lag_30,sales_lag_31,sales_lag_32,sales_lag_33,sales_lag_34,sales_lag_35,sales_lag_36,sales_lag_37,sales_lag_38,sales_lag_39,sales_lag_40,sales_lag_41,sales_lag_42,rolling_mean_7,rolling_std_7,rolling_mean_14,rolling_std_14,rolling_mean_30,rolling_std_30,rolling_mean_60,rolling_std_60,rolling_mean_180,rolling_std_180,store_id,rolling_mean_tmp_1_7,rolling_mean_tmp_1_14,rolling_mean_tmp_1_30,rolling_mean_tmp_1_60,rolling_mean_tmp_7_7,rolling_mean_tmp_7_14,rolling_mean_tmp_7_30,rolling_mean_tmp_7_60,rolling_mean_tmp_14_7,rolling_mean_tmp_14_14,rolling_mean_tmp_14_30,rolling_mean_tmp_14_60
3902435,FOODS_3_823_WI_3_validation,1941,NaN,FOODS_3_823,FOODS_3,FOODS,0,2.980469,2.980469,2.480469,0.171631,2.800781,1.000000,5.0,206,1.0,1.029297,1.022461,NaN,NaN,NaN,NaN,0,0,0,22,20,5,5,4,6,1,2.109375,5.769531,2.626953,7.058594,0.846680,1.752930,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.285645,0.488037,0.428467,7.558594e-01,0.199951,0.550781,0.250000,0.571289,0.616699,0.987305,WI_3,0.979912,0.979654,0.910022,0.538344,0.978399,0.978168,0.747475,0.440404,0.977937,0.969676,0.652515,0.326258
3902436,FOODS_3_824_WI_3_validation,1941,NaN,FOODS_3_824,FOODS_3,FOODS,0,2.480469,2.679688,2.000000,0.253174,2.507812,0.925293,4.0,135,1.0,0.997070,1.112305,NaN,NaN,NaN,NaN,0,0,0,22,20,5,5,4,6,1,2.109375,5.769531,2.626953,7.058594,0.434326,0.947266,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.142822,0.377930,0.214233,4.257812e-01,0.300049,0.535156,0.150024,0.404541,0.049988,0.242798,WI_3,0.970469,0.973769,0.905550,0.586108,0.976190,0.971974,0.711585,0.489126,0.967759,0.965301,0.617140,0.375237
3902437,FOODS_3_825_WI_3_validation,1941,NaN,FOODS_3_825,FOODS_3,FOODS,0,3.980469,4.378906,3.980469,0.188599,4.117188,0.908691,3.0,150,1.0,0.965820,1.000000,NaN,NaN,NaN,NaN,0,0,0,22,20,5,5,4,6,1,2.109375,5.769531,2.626953,7.058594,0.700684,1.198242,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,2.0,0.0,0.0,0.0,4.0,2.0,0.571289,0.786621,0.785645,1.188477e+00,0.866699,1.136719,1.033203,1.056641,0.761230,0.941895,WI_3,0.985841,0.992865,0.927696,0.913848,0.999112,0.999460,0.864487,0.898910,0.999807,0.995772,0.964694,0.949014
3902438,FOODS_3_826_WI_3_validation,1941,NaN,FOODS_3_826,FOODS_3,FOODS,230,1.280273,1.280273,1.280273,0.000000,1.280273,1.000000,1.0,44,1.0,1.000000,1.000000,NaN,NaN,NaN,NaN,0,0,0,22,20,5,5,4,6,1,2.109375,5.769531,2.626953,7.058594,1.108398,1.479492,3.0,1.0,3.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0,1.0,0.0,2.0,1.142578,1.344727,0.928711,1.207031e+00,1.066406,1.172852,1.016602,1.065430,1.338867,1.550781,WI_3,0.994172,1.000594,1.134251,1.050459,1.006245,1.007142,1.002575,1.034621,1.008040,1.002395,1.001118,1.050559
3902439,FOODS_3_827_WI_3_validation,1941,NaN,FOODS_3_827,FOODS_3,FOODS,304,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.0,142,1.0,1.000000,1.000000,NaN,NaN,NaN,NaN,0,0,0,22,20,5,5,4,6,1,2.109375,5.769531,2.626953,7.058594,1.644531,3.115234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,1.192093e-07,1.166992,1.821289,1.500000,1.770508,1.510742,1.741211,WI_3,1.003083,1.007752,0.895703,1.197852,1.013276,0.998230,0.695702,1.264517,0.983184,0.984151,0.492604,1.396302


In [0]:
gc.collect()

In [0]:
END_TRAIN-100

1813

In [0]:
! mv /content/gdrive/"My Drive"/kaggle//output/m5modelwithestimator1300/*pkl /content/

'My Drive'
